In [1]:
import detect
import shutil
import cv2
import numpy as np
import os
import pymongo

C:\Users\gbuln\anaconda3\envs\luna\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client['Yeji']
collection3 = db['Resident']


In [14]:
# 정기권 차량1등록
document1 = {
    'carnum': '02나4287',
    'registered': True,
    'imageURL': 'C:\\Users\\gbuln\\Downloads\\fakenumber\\02나4287_in_473947e7-3540-4169-98fa-7f7743ae3ed1.jpg'
}
collection3.insert_one(document1)

In [15]:
#정기권차량2등록
document2 = {
    'carnum': '01누0599',
    'registered': True,
    'imageURL': 'C:\\Users\\gbuln\\Downloads\\fakenumber\\01누0599_in_43f75ed2-77a7-446c-b4c3-314f73940400.jpg'
}
collection3.insert_one(document2)

In [4]:
#입출차기록 db에 삽입
# collection = db.create_collection('InAndOut')
collection = db['InAndOut']

In [30]:
# 정상차량 등장()

document3 = {
    'accessid' : 3,
    'intime': '2023-09-27 23:22:22',
    'outtime' : '',
    'carnum': '01누0599',
    'faked': False,
    'imageURL': 'C:\\Users\\gbuln\\Downloads\\fakenumber\\01누0599_in_9d6eda83-f0aa-4d55-9595-34e1ff43f4c1.jpg'
}

In [13]:
#입출차기록에 넣어준다.
collection = db["InAndOut"]
collection.insert_one(document3)

In [31]:
import pymongo

# Connect to the MongoDB server
client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client['Yeji']
# Get the Resident collection
collection = db["Resident"]

# Find the document with carnum "01누0599" 01누0599인 차량을 찾는다.
cursor = collection.find({"carnum": document3['carnum']})

# Print the imageURL field of the document
for document in cursor:
    print(document["imageURL"])
    original_path = document["imageURL"] #원본 사진의 url 주소


C:\Users\gbuln\Downloads\fakenumber\01누0599_in_43f75ed2-77a7-446c-b4c3-314f73940400.jpg


In [32]:
# source_path=original_path

In [33]:
save_path = f"C:\\Users\\gbuln\\Downloads\\{document3['carnum']}" #차량번호를 이름으로 한 폴더를 만들어준다.
if not os.path.exists(save_path):
    os.makedirs(save_path)

In [34]:


#  헤드라이트만 크롭하기 위해 입출차내역 사진부터 옮겨주고,
src = document3["imageURL"]

# The destination directory for the copy
dst = save_path

# Copy the file
shutil.copy(src, dst)


'C:\\Users\\gbuln\\Downloads\\01누0599\\01누0599_in_9d6eda83-f0aa-4d55-9595-34e1ff43f4c1.jpg'

In [35]:

# 헤드라이트 크롭을 위해 원본의 사진도 옮겨준다(나중에 미리 크롭해뒀다가 db에 저장해두는게 빠를지도?)
src = original_path

# The destination directory for the copy
dst = save_path

# Copy the file
shutil.copy(src, dst)

'C:\\Users\\gbuln\\Downloads\\01누0599\\01누0599_in_43f75ed2-77a7-446c-b4c3-314f73940400.jpg'

In [36]:
#detect 하는 코드
detect.run(weights=".\\headlights_v5.pt", 
                source=save_path,
                conf_thres=0.7,
                save_crop=True,
                save_txt=True,
                save_conf=True)

YOLOv5  v6.1-306-gfbe67e4 Python-3.9.15 torch-1.13.1+cpu CPU

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients
image 1/2 C:\Users\gbuln\Downloads\01누0599\01누0599_in_43f75ed2-77a7-446c-b4c3-314f73940400.jpg: 480x640 1 headlight, Done. (0.170s)
image 2/2 C:\Users\gbuln\Downloads\01누0599\01누0599_in_9d6eda83-f0aa-4d55-9595-34e1ff43f4c1.jpg: 480x640 1 headlight, Done. (0.164s)
Speed: 0.5ms pre-process, 167.0ms inference, 0.5ms NMS per image at shape (1, 3, 640, 640)
Results saved to runs\detect\exp23
2 labels saved to runs\detect\exp23\labels


In [37]:
# 디렉토리에 있는 jpg 파일명을 한꺼번에 리스트에 추가해 주는 코드 
# 파일 이름 확인용으로 그냥 짰다.

root_path = '.\\runs\\detect\\exp23\\crops\\headlight'

In [38]:
# get a list of all files in the directory
files = os.listdir(root_path)

# create an empty list to store filenames
jpg_filenames = []

# iterate over the files
for file in files:
  # check if the file is a JPG file
  if file.endswith('.jpg'):
    # remove the '.jpg' from the filename
    filename = file[:-4]
    # add the filename to the list
    jpg_filenames.append(filename)

# print the filenames
print(jpg_filenames)

['01누0599_in_43f75ed2-77a7-446c-b4c3-314f73940400', '01누0599_in_9d6eda83-f0aa-4d55-9595-34e1ff43f4c1']


In [39]:
# 매칭점 찾기 코드는 한글을 못읽어서 죄다 영어로 바꿔줘야 한다. 그 전처리 과정.
import os
# '-'를 '_'f로 바꿔주는 코드
## Directory where the files are located
directory = root_path

# Get a list of all files in the directory
files = os.listdir(directory)

# Loop through the files in the directory
for file in files:
    # Check if the file name contains "-"
    if "누" in file:
        # Get the file's old name and new name (with "-" replaced with "_")
        old_name = os.path.join(directory, file)
        new_name = os.path.join(directory, file.replace("누", "nu"))

        # Use the rename function to rename the file
        os.rename(old_name, new_name)


In [40]:
# 매칭점 찾는 코드 (서브프로세스를 써서 에러가 났는지 안났는지 잘 모름. 결과물인 matches.txt가 output_dir에 있어야 함)
import subprocess

input_dir = root_path #사진 저장된 디렉토리(여기서는 욜로의 detect한 path)
output_dir = save_path #매칭첨 개수가 저장된 txt가 저장될 폴더(어차피 save_path가 지워지므로 그냥 여기다가 저장했다.)
resize = "320 240"
no_display = True
max_keypoints = 2048
nms_radius = 3
keypoint_threshold = 0.1
match_threshold = 0.3

command = f"python ./demo_superglue.py --input {input_dir} --output_dir {output_dir} --resize {resize}"

if no_display:
    command += " --no_display"

command += f" --max_keypoints {max_keypoints} --nms_radius {nms_radius} --keypoint_threshold {keypoint_threshold} --match_threshold {match_threshold}"

subprocess.run(command.split())

CompletedProcess(args=['python', './demo_superglue.py', '--input', '.\\runs\\detect\\exp23\\crops\\headlight', '--output_dir', 'C:\\Users\\gbuln\\Downloads\\01누0599', '--resize', '320', '240', '--no_display', '--max_keypoints', '2048', '--nms_radius', '3', '--keypoint_threshold', '0.1', '--match_threshold', '0.3'], returncode=0)

In [41]:
# 에러난것 같으면 아래 코드 주석지우고 돌려보면 된다. 여기서 무슨 에러가 떴는지 잡힘
# !python ./demo_superglue.py --input ./runs/detect/exp21/crops/headlight --output_dir beo01 --resize 320 240 --no_display --max_keypoints 2048 --nms_radius 3 --keypoint_threshold 0.1 --match_threshold 0.3

In [42]:
with open(output_dir + '\\matches.txt', 'r') as f:
    content = f.read()
print(content)

77


In [43]:
collection = db["InAndOut"]
if int(content) < 20:
    collection.update_one({"accessid": 1}, {"$set": {"faked": True}})
else:
    pass

In [44]:

cursor = db['InAndOut'].find({"accessid": document3['accessid']})
for document in cursor:
    print(document)

{'_id': ObjectId('63bcbd728d9eef5f7166dca3'), 'accessid': 3, 'intime': '2023-09-27 23:22:22', 'outtime': '', 'carnum': '01누0599', 'faked': False, 'imageURL': 'C:\\Users\\gbuln\\Downloads\\fakenumber\\01누0599_in_9d6eda83-f0aa-4d55-9595-34e1ff43f4c1.jpg'}


In [45]:
shutil.rmtree('.\\runs\\detect\\exp23')
shutil.rmtree(save_path)